In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from bigfile import BigFile
import glob,os,struct
# from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from scipy.interpolate import interp1d
from colossus.cosmology import cosmology

import seaborn as sns

sns.set()
sns.set_palette("Set2")
sns.set_style('ticks',{'ytick.direction':'in','xtick.direction':'in'})

cmap = plt.get_cmap("Set2")
sns.set_context("paper", font_scale=1.7,rc={"axes.linewidth": 1.3,"lines.linewidth": 2.5,"patch.linewidth": 2.2})
from matplotlib import rcParams as rc
import pickle
import warnings
import h5py

In [2]:
snap = 147

subroot = '/hildafs/datasets/Asterix//subfind/subfind_%03d'%snap
tab="fof_subhalo_tab_%03d.hdf5"%snap
grp = "snap-groupordered_%03d.hdf5"%snap

In [5]:
def get_subfind_chunk(subroot):
    subdir = subroot + '/chunk*'
    chunk_list    = []
    maxgroup_list = []
    for ff in sorted(glob.glob(subdir)):
        cname    = ff.split('/')[-1]
        chunk    = int(cname.split('.')[0][5:])
        maxgroup = int(cname.split('.')[1])
        
        chunk_list.append(chunk)
        maxgroup_list.append(maxgroup)
        
    sort  = np.argsort(chunk_list)
    chunk_list    = np.array(chunk_list)[sort]
    maxgroup_list = np.array(maxgroup_list)[sort]
    return chunk_list, maxgroup_list 

In [6]:
chunk_list, maxgroup_list = get_subfind_chunk(subroot)
len(chunk_list)
chunks = chunk_list

In [15]:
cspec = []
for c,m in zip(chunks, maxgroup_list):
    if os.path.isdir(subroot + '/chunk%d.%d/output/groups_%d'%(c,m,snap)):
        cspec.append(subroot + '/chunk%d.%d/output'%(c,m))

In [19]:
ff = cspec[0]

tabfiles = sorted(glob.glob(ff + '/groups_%03d/fof_subhalo_tab_%03d.*.hdf5'%(snap, snap)))
print(tabfiles)

['/hildafs/datasets/Asterix//subfind/subfind_147/chunk381.59662736/output/groups_147/fof_subhalo_tab_147.0.hdf5', '/hildafs/datasets/Asterix//subfind/subfind_147/chunk381.59662736/output/groups_147/fof_subhalo_tab_147.1.hdf5']


In [20]:
off1 = h5py.File(tabfiles[0],'r')['Subhalo']['SubhaloOffsetType'][:]
off2 = h5py.File(tabfiles[1],'r')['Subhalo']['SubhaloOffsetType'][:]

In [30]:
h5py.File(tabfiles[0],'r')['Header'].attrs['Nsubhalos_ThisFile']

635894

In [31]:
h5py.File(tabfiles[1],'r')['Header'].attrs['Nsubhalos_ThisFile']

635469